In [1]:
import geopandas as gpd
import pandas as pd

cap_dict = {}
#input
# CLASS,L1,L2,L3,L4,L5,L6
# 1, 2300, 4300, 6300, 8300, 10300, 12300
#disct: (class,lanes)=capacity eg (39,6)=9420
with open('TRTS4S_Capacity.csv', 'r') as f:
    lines = f.readlines()
    for line in lines[1:]:  # Skip the header line
        parts = line.strip().split(',')
        class_num = int(parts[0])
        for i in range(1, len(parts)):
            lanes = i
            capacity = int(parts[i])
            cap_dict[(class_num, lanes)] = capacity

In [7]:
# (編修link)
# Net_modified.shp
# Net_modified_全線拆除.shp
# Net_modified_部分拆除.shp

# (調撥後容量調整)
# Net_modified_晨峰調撥
# Net_modified_全線拆除_晨峰調撥
# Net_modified_部分拆除_晨峰調撥
# Net_modified_昏峰調撥
# Net_modified_全線拆除_昏峰調撥
# Net_modified_部分拆除_昏峰調撥 

scenario = '_全線拆除'
# scenario = '_部分拆除'

In [8]:
#調撥車道處理(晨峰)
#no need to look up capacity
gdf = gpd.read_file(f'Net_modified{scenario}.shp')

# 承德路7:00-9:00
# [8299,6666]:(5,3) means (A,B)=(8299,6666), LANES_110=5; (A,B)=(6666,8299), LANES_110=3
# [6666,6628,8293,15665,9978,8289]:(4,2)
# [6631,6660,6642,8141,14039,8143,9883,6387]:(5,3)
# 中山北路7:00-9:00
# [6291,6696,6695,6629,8291]:(3,1)
# 民族東路7:00-9:00
# [8345,6682,9154,15473,6375,6683]:(3,1)
# [6683,6373,6409,6398]:(4,2)
# 復興北路 五常街至民權東路 07:00~09:00
# [6753,8090]:(4,2)
# 北安路 大直橋至大直街 07:00-09:00
# [6812,16591]:(2,4)
# 新生南路 辛亥路至和平東路 07:00~09:00
# [5994,15176,6018]:(3,5)
# 環河南路 開封街至忠孝西路 07:00~09:00
# [8156,5303]:(3,1)
# 重慶南路 中正橋頭至愛國西路 07:00~09:00
# [9760,27354,9758,6085,6092,6091,6086]:(4,2)
# [6086,6065]:(3,1)
# [6065,6076]:(5,3)
# [9060,15387,6093,8401,6076]:(2,4)
# 水源路 水源路35巷至師大路口 07:00~09:00 17:00~19:00
# [6056,15347]:(0,2)
# 永福橋 永福橋頭至自來水南區營業處 07:00~09:00
# [10016,9796]:(2,0)
# [9796,27340,6104]:(3,1)
# [6104,6102,6103]:(2,0)
# 敦化南路 基隆路至仁愛路 07:00~09:00
# [7736,7836,7833,5323,7834,7835]:(0,3)
# [7835,6227,6224]:(1,3)
# 南港路 南港橋頭至研究院路 07:00~09:00
# [7037,15826,7025,7023,9259,16385,7045,10251,7046,16384,9270,7050]:(2,4)
# 研究院路 南港水廠至忠孝東路 07:00~09:00
# [7028,15831,8685,7029,15833]:(2,4)
# 民權東路 國醫中心至成功路 07:00~09:00
# [6904,15761,6874]:(2,4)

#中山北路酒泉街至民生東路
# [6676,6392,9218,9219,9121,8108,6393,8107,8071,8079]:3
# [8361,8357,8354,8351,8348,9220,9217,8344,6676]:1

reversible_links_dict = {
    (8299, 6666): (5, 3),
    (6666, 6628, 8293, 15665, 9978, 8289): (4, 2),
    (6631, 6660, 6642, 8141, 14039, 8143, 9883, 6387): (5, 3),
    (6291, 6696, 6695, 6629, 8291): (3, 1),
    (8345, 6682, 9154, 15473, 6375, 6683): (3, 1),
    (6683, 6373, 6409, 6398): (4, 2),
    (6753, 8090): (4, 2),
    (6812,16591):(2,4),
    (5994, 15176, 6018): (3, 5),
    (8156, 5303): (3, 1),
    (9760, 27354, 9758, 6085, 6092, 6091, 6086): (4, 2),
    (6086, 6065): (3, 1),
    (6065, 6076): (5, 3),
    (9060, 15387, 6093, 8401, 6076): (2, 4),
    (6056, 15347): (0, 2),
    (10016, 9796): (2, 0),
    (9796, 27340, 6104): (3, 1),
    (6104, 6102, 6103): (2, 0),
    (7736, 7836, 7833, 5323, 7834, 7835): (0, 3),
    (7835, 6227, 6224): (1, 3),
    (7037, 15826, 7025, 7023, 9259, 16385,7045 ,10251 ,7046 ,16384 ,9270 ,7050): (2 ,4),
    (7028 ,15831 ,8685 ,7029 ,15833): (2 ,4),
    (6904 ,15761 ,6874): (2 ,4),
    (6676, 6392, 9218, 9219, 9121, 8108, 6393, 8107, 8071, 8079): (3, 3),  # 中山北路
    (8361, 8357, 8354, 8351, 8348, 9220, 9217, 8344, 6676): (1, 1)  # 中山北路
}


# 將可調撥車道轉換為字典形式
reversible_links = {}
for key, value in reversible_links_dict.items():
    if isinstance(key, tuple):
        for i in range(len(key) - 1):
            reversible_links[(key[i], key[i + 1])] = value[0]  # 使用第一個值作為車道數
            reversible_links[(key[i + 1], key[i])] = value[1]  # 使用第二個值作為車道數
    else:
        reversible_links[key] = value[0]  # 單一連結的情況
        reversible_links[(key[1], key[0])] = value[1]  # 反向連結

print("Reversible links:", reversible_links)

# 遍歷每一行數據
for index, row in gdf.iterrows():
    # 獲取 A 和 B 的值
    A = row['A']
    B = row['B']
    
    # 獲取車道數
    lanes = row['LANES_110']
    
    # 獲取 CLASS_110
    class_num = row['CLASS_110']
    
    # 檢查是否為可調撥車道
    if (A, B) in reversible_links:
        lanes = reversible_links[(A, B)]
        capacity = cap_dict.get((class_num, lanes), 0)
    elif (B, A) in reversible_links:
        lanes = reversible_links[(B, A)]
        capacity = cap_dict.get((class_num, lanes), 0)
    else:
        capacity = cap_dict.get((class_num, lanes), 0)
    
    
    # 更新行數據(LANES_110, capacity)
    gdf.at[index, 'LANES_110'] = lanes
    gdf.at[index, 'capacity'] = capacity

# 保存修改後的 GeoDataFrame
gdf.to_file(f'Net_modified{scenario}_晨峰調撥.shp', driver='ESRI Shapefile')
#gdf to csv
gdf.to_csv(f'Net_modified{scenario}_晨峰調撥.csv', index=False)

Reversible links: {(8299, 6666): 5, (6666, 8299): 3, (6666, 6628): 4, (6628, 6666): 2, (6628, 8293): 4, (8293, 6628): 2, (8293, 15665): 4, (15665, 8293): 2, (15665, 9978): 4, (9978, 15665): 2, (9978, 8289): 4, (8289, 9978): 2, (6631, 6660): 5, (6660, 6631): 3, (6660, 6642): 5, (6642, 6660): 3, (6642, 8141): 5, (8141, 6642): 3, (8141, 14039): 5, (14039, 8141): 3, (14039, 8143): 5, (8143, 14039): 3, (8143, 9883): 5, (9883, 8143): 3, (9883, 6387): 5, (6387, 9883): 3, (6291, 6696): 3, (6696, 6291): 1, (6696, 6695): 3, (6695, 6696): 1, (6695, 6629): 3, (6629, 6695): 1, (6629, 8291): 3, (8291, 6629): 1, (8345, 6682): 3, (6682, 8345): 1, (6682, 9154): 3, (9154, 6682): 1, (9154, 15473): 3, (15473, 9154): 1, (15473, 6375): 3, (6375, 15473): 1, (6375, 6683): 3, (6683, 6375): 1, (6683, 6373): 4, (6373, 6683): 2, (6373, 6409): 4, (6409, 6373): 2, (6409, 6398): 4, (6398, 6409): 2, (6753, 8090): 4, (8090, 6753): 2, (6812, 16591): 2, (16591, 6812): 4, (5994, 15176): 3, (15176, 5994): 5, (15176, 6018)

c:\Users\Allen\anaconda3\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [9]:
# Network

input_file = f'Net_modified{scenario}_晨峰調撥.csv'
output_file = f'Net_modified{scenario}_晨峰調撥.dat'

df = pd.read_csv(input_file)
# if there is '', then replace with 0
df_out = pd.DataFrame(columns=['origin','dest','capacity','length','fft','alpha','beta','speedLimit'])
df.fillna(0, inplace=True)
rows = []

for index, row in df.iterrows():
    if row["LEVEL_110"] in [1,2,3,4,5,6,7,11]:# and row["CLASS_110"] #not in [i for i in range(84, 99)]+[61]:
        # if DIR=1, then A is origin, B is dest
        # if DIR=-1, then A is dest, B is origin
        # if DIR=2, then the road is bidirectional, so we need to add two rows
        # if row["DIR"] == 1 or row["DIR"] == 2:
            # print(row["ID"], row["DIR"], row["A"], row["B"])
        origin = row["A"]
        dest = row["B"]
        # elif row["DIR"] == -1:
            # print(row["ID"], row["DIR"], row["A"], row["B"])
            # origin = row["B"]
            # dest = row["A"]
        capacity = cap_dict.get((row["CLASS_110"], row["LANES_110"]), 0)
        if row["CLASS_110"] == 99:
            capacity = 9999
        #4359,8218,8251,8252,3677,8260,10923,8270,8271,10928,3646,4018 ID, half capacity
        if row["ID"] in [4359, 8218, 8251, 8252, 3677, 8260, 10923, 8270, 8271, 10928, 3646, 4018]:
            capacity = capacity/2
        #2343 ID, 0.85 capacity
        if row["ID"] == 2343:
            capacity = capacity*0.85
        length = float(row["LENGTH"])  or 1000
        try:
            fft = length/float(row["S0"])*60
        except:
            # if ID is not 40438, then give S0=32
            # print(row["ID"], row["S0"])
            if row["ID"] != 40438:
                fft = length/32*60
            else:
                fft = 1000
        alpha = row["ALPHA"] or 0
        beta = row["BETA"] or 0
        speedLimit = 0
        if capacity == 0:
            continue
        rows.append({'origin':origin,'dest':dest,'capacity':capacity,'length':length,'fft':fft,'alpha':alpha,'beta':beta,'speedLimit':speedLimit})
        # if row["DIR"] == 2:
            # add the reverse direction
            # rows.append({'origin':dest,'dest':origin,'capacity':capacity,'length':length,'fft':fft,'alpha':alpha,'beta':beta,'speedLimit':speedLimit})

# write in df_supplement from network_supplement.dat
df_supplement = pd.read_csv('network_supplement.dat', sep='\t', header=None, names=['origin','dest','capacity','length','fft','alpha','beta','speedLimit'])
# set columns=['origin','dest','capacity','length','fft','alpha','beta','speedLimit']

# tail.node	head.node	capacity..veh.h.		fftt.min.			speed.limit..mph.

df_out = pd.concat([df_out, pd.DataFrame(rows)], ignore_index=True)
df_out = pd.concat([df_out, df_supplement], ignore_index=True)

df_out.to_csv(output_file, index=False, sep='\t')

C:\Users\Allen\AppData\Local\Temp\ipykernel_12472\3891746405.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_out = pd.concat([df_out, pd.DataFrame(rows)], ignore_index=True)


In [10]:
#調撥車道處理(昏峰)
#no need to look up capacity
#open Y110_modified_v8.shp
gdf = gpd.read_file(f'Net_modified{scenario}.shp')

# 水源路 水源路35巷至師大路口 07:00~09:00 17:00~19:00
# [6056,15347]:(0,2)
# 舊宗路 行善路至南京東路6段 17:30~19:30
# [6962,15776,7986]:(4,2)
# 瑞湖街 舊宗路2段171巷至民權東路6段11巷 17:00~19:00
# [7013,15768,7008,6910]:(2,0)
# 民生西路 迪化街至環河北路 16:00~19:00
# [6548,9106,6551,8057]:(1,3)
# 基隆路 臺北市調處至基隆高架道路樂業匝道 17:00~19:30
# [6225,6228,6205,15323,6194,15321]:(2,4)
# [6201,6225]:swap A B

reversible_links_dict = {
    (6056, 15347): (0, 2),
    (6962, 15776, 7986): (4, 2),
    (7013, 15768, 7008, 6910): (2, 0),
    (6548, 9106, 6551, 8057): (1, 3),
    (6225, 6228, 6205, 15323, 6194, 15321): (2, 4)
}
    

# 將可調撥車道轉換為字典形式
reversible_links = {}
for key, value in reversible_links_dict.items():
    if isinstance(key, tuple):
        for i in range(len(key) - 1):
            reversible_links[(key[i], key[i + 1])] = value[0]  # 使用第一個值作為車道數
            reversible_links[(key[i + 1], key[i])] = value[1]  # 使用第二個值作為車道數
    else:
        reversible_links[key] = value[0]  # 單一連結的情況
        reversible_links[(key[1], key[0])] = value[1]  # 反向連結

print("Reversible links:", reversible_links)

# 遍歷每一行數據
for index, row in gdf.iterrows():
    # 獲取 A 和 B 的值
    A = row['A']
    B = row['B']
    
    # 獲取車道數
    lanes = row['LANES_110']
    
    # 獲取 CLASS_110
    class_num = row['CLASS_110']
    
    # 檢查是否為可調撥車道
    if (A, B) in reversible_links:
        lanes = reversible_links[(A, B)]
        capacity = cap_dict.get((class_num, lanes), 0)
    elif (B, A) in reversible_links:
        lanes = reversible_links[(B, A)]
        capacity = cap_dict.get((class_num, lanes), 0)
    elif (A, B) == (6201, 6225):
        # 特例處理，A和B互換，車道數不變
        gdf.at[index, 'A'] = B
        gdf.at[index, 'B'] = A
    else:
        capacity = cap_dict.get((class_num, lanes), 0)
    
    # 更新行數據(LANES_110, capacity)
    gdf.at[index, 'LANES_110'] = lanes
    gdf.at[index, 'capacity'] = capacity

# 保存修改後的 GeoDataFrame
gdf.to_file(f'Net_modified{scenario}_昏峰調撥.shp', driver='ESRI Shapefile')
#gdf to csv
gdf.to_csv(f'Net_modified{scenario}_昏峰調撥.csv', index=False)

Reversible links: {(6056, 15347): 0, (15347, 6056): 2, (6962, 15776): 4, (15776, 6962): 2, (15776, 7986): 4, (7986, 15776): 2, (7013, 15768): 2, (15768, 7013): 0, (15768, 7008): 2, (7008, 15768): 0, (7008, 6910): 2, (6910, 7008): 0, (6548, 9106): 1, (9106, 6548): 3, (9106, 6551): 1, (6551, 9106): 3, (6551, 8057): 1, (8057, 6551): 3, (6225, 6228): 2, (6228, 6225): 4, (6228, 6205): 2, (6205, 6228): 4, (6205, 15323): 2, (15323, 6205): 4, (15323, 6194): 2, (6194, 15323): 4, (6194, 15321): 2, (15321, 6194): 4}


c:\Users\Allen\anaconda3\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [11]:
# Network

input_file = f'Net_modified{scenario}_昏峰調撥.csv'
output_file = f'Net_modified{scenario}_昏峰調撥.dat'

df = pd.read_csv(input_file)
# if there is '', then replace with 0
df_out = pd.DataFrame(columns=['origin','dest','capacity','length','fft','alpha','beta','speedLimit'])
df.fillna(0, inplace=True)
rows = []

for index, row in df.iterrows():
    if row["LEVEL_110"] in [1,2,3,4,5,6,7,11]:# and row["CLASS_110"] #not in [i for i in range(84, 99)]+[61]:
        # if DIR=1, then A is origin, B is dest
        # if DIR=-1, then A is dest, B is origin
        # if DIR=2, then the road is bidirectional, so we need to add two rows
        # if row["DIR"] == 1 or row["DIR"] == 2:
            # print(row["ID"], row["DIR"], row["A"], row["B"])
        origin = row["A"]
        dest = row["B"]
        # elif row["DIR"] == -1:
            # print(row["ID"], row["DIR"], row["A"], row["B"])
            # origin = row["B"]
            # dest = row["A"]
        capacity = cap_dict.get((row["CLASS_110"], row["LANES_110"]), 0)
        if row["CLASS_110"] == 99:
            capacity = 9999
        #4359,8218,8251,8252,3677,8260,10923,8270,8271,10928,3646,4018 ID, half capacity
        if row["ID"] in [4359, 8218, 8251, 8252, 3677, 8260, 10923, 8270, 8271, 10928, 3646, 4018]:
            capacity = capacity/2
        #2343 ID, 0.85 capacity
        if row["ID"] == 2343:
            capacity = capacity*0.85
        length = float(row["LENGTH"])  or 1000
        try:
            fft = length/float(row["S0"])*60
        except:
            # if ID is not 40438, then give S0=32
            # print(row["ID"], row["S0"])
            if row["ID"] != 40438:
                fft = length/32*60
            else:
                fft = 1000
        alpha = row["ALPHA"] or 0
        beta = row["BETA"] or 0
        speedLimit = 0
        if capacity == 0:
            continue
        rows.append({'origin':origin,'dest':dest,'capacity':capacity,'length':length,'fft':fft,'alpha':alpha,'beta':beta,'speedLimit':speedLimit})
        # if row["DIR"] == 2:
            # add the reverse direction
            # rows.append({'origin':dest,'dest':origin,'capacity':capacity,'length':length,'fft':fft,'alpha':alpha,'beta':beta,'speedLimit':speedLimit})

# write in df_supplement from network_supplement.dat
df_supplement = pd.read_csv('network_supplement.dat', sep='\t', header=None, names=['origin','dest','capacity','length','fft','alpha','beta','speedLimit'])
# set columns=['origin','dest','capacity','length','fft','alpha','beta','speedLimit']

# tail.node	head.node	capacity..veh.h.		fftt.min.			speed.limit..mph.

df_out = pd.concat([df_out, pd.DataFrame(rows)], ignore_index=True)
df_out = pd.concat([df_out, df_supplement], ignore_index=True)

df_out.to_csv(output_file, index=False, sep='\t')

C:\Users\Allen\AppData\Local\Temp\ipykernel_12472\1154423199.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_out = pd.concat([df_out, pd.DataFrame(rows)], ignore_index=True)
